**Importing the needed packages**

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from deap import base, creator, tools, algorithms
import lightgbm as lgb
import random

In [6]:
df_monday = pd.read_csv("C:/VS code projects/data_files/Monday-WorkingHours.pcap_ISCX.csv")
df_tuesday = pd.read_csv("C:/VS code projects/data_files/Tuesday-WorkingHours.pcap_ISCX.csv")
df_wednesday = pd.read_csv("C:/VS code projects/data_files/Wednesday-workingHours.pcap_ISCX.csv")
df_thursday_afternoon = pd.read_csv("C:/VS code projects/data_files/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df_thursday_morning= pd.read_csv("C:/VS code projects/data_files/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df_friday_afternoon_ddos = pd.read_csv("C:/VS code projects/data_files/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df_friday_afternoon_portscan = pd.read_csv("C:/VS code projects/data_files/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df_friday_morning = pd.read_csv("C:/VS code projects/data_files/Friday-WorkingHours-Morning.pcap_ISCX.csv")
df_monday.columns = df_monday.columns.str.strip()
df_tuesday.columns = df_tuesday.columns.str.strip()
df_wednesday.columns = df_wednesday.columns.str.strip()
df_thursday_afternoon.columns = df_thursday_afternoon.columns.str.strip()
df_thursday_morning.columns = df_thursday_morning.columns.str.strip()
df_friday_afternoon_ddos.columns = df_friday_afternoon_ddos.columns.str.strip()
df_friday_afternoon_portscan.columns = df_friday_afternoon_portscan.columns.str.strip()
df_friday_morning.columns = df_friday_morning.columns.str.strip()

In [7]:
df = pd.concat([df_monday,df_tuesday,df_wednesday, df_thursday_afternoon, df_thursday_morning, df_friday_afternoon_ddos, df_friday_afternoon_portscan, df_friday_morning], ignore_index=True)
df['Label'] = df['Label'].apply(lambda x: 1 if x == 'BENIGN' else 0)

# Find the minimum class count
min_count = df['Label'].value_counts().min()


# Perform undersampling to balance the dataset
df = df.groupby('Label', group_keys=False).apply(lambda x: x.sample(n=min_count, random_state=42, ignore_index=True))

# Check the class distribution
df['Label'].value_counts()


C:\Users\Spencer Kanjera\AppData\Local\Temp\ipykernel_25876\2654131954.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Label', group_keys=False).apply(lambda x: x.sample(n=min_count, random_state=42, ignore_index=True))


Label
0    557646
1    557646
Name: count, dtype: int64

In [8]:
# Preprocessing dataset
print("Preprocessing dataset...")

# Step 1: Replace infinite values with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Step 2: Handle NaN values (e.g., fill with 0 or column mean)
df.fillna(0, inplace=True)

print("Done preprocessing dataset")

Preprocessing dataset...
Done preprocessing dataset


In [9]:
# Step 3: Encode categorical labels
#label_encoder = LabelEncoder()

#df['Label'] = label_encoder.fit_transform(df['Label'])
#print("Labels encoded successfully!")
df['Label'].value_counts()

Label
0    557646
1    557646
Name: count, dtype: int64

In [10]:
# Step 4: Separate features and labels
X = df.drop('Label', axis=1)
Y = df['Label']

In [11]:
# Step 5: Normalize numerical features using MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [12]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [13]:
# Filter Method: Calculate correlations between features and labels
print("Calculating feature importance using correlation...")
correlations = []
for i in range(X_train.shape[1]):
    if np.std(X_train[:, i]) == 0:  # Skip constant features
        correlations.append(0)
    else:
        correlations.append(abs(pearsonr(X_train[:, i], y_train)[0]))
correlation_threshold = 0.2  # Define a threshold to filter irrelevant features
relevant_features = [i for i, corr in enumerate(correlations) if corr > correlation_threshold]
print(f"Features with correlation above threshold: {len(relevant_features)}")

# Subset the data with relevant features only
X_train = X_train[:, relevant_features]
X_test = X_test[:, relevant_features]
feature_names = [df.columns[i] for i in relevant_features]
[print(f"{i}: {feature_names[i]}") for i in range(len(feature_names))]

Calculating feature importance using correlation...
Features with correlation above threshold: 23
0: Destination Port
1: Flow Duration
2: Bwd Packet Length Max
3: Bwd Packet Length Min
4: Bwd Packet Length Mean
5: Bwd Packet Length Std
6: Flow IAT Std
7: Flow IAT Max
8: Fwd IAT Total
9: Fwd IAT Std
10: Fwd IAT Max
11: Min Packet Length
12: Max Packet Length
13: Packet Length Mean
14: Packet Length Std
15: Packet Length Variance
16: PSH Flag Count
17: URG Flag Count
18: Average Packet Size
19: Avg Bwd Segment Size
20: Idle Mean
21: Idle Max
22: Idle Min


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [14]:
# GA Feature Selection
print("Defining Genetic Algorithm for feature selection...")

# Define GA components
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=X_train.shape[1])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

Defining Genetic Algorithm for feature selection...


In [15]:
# Define the fitness function
def fitness(individual, X_train, y_train):
    selected_features = [i for i, bit in enumerate(individual) if bit == 1]
    if len(selected_features) == 0:  # Avoid empty feature sets
        return 0,
    X_train_selected = X_train[:, selected_features]
    model = lgb.LGBMClassifier(random_state=42)
    model.fit(X_train_selected, y_train)
    accuracy = model.score(X_train_selected, y_train)  # Training accuracy
    return accuracy,

toolbox.register("evaluate", fitness, X_train=X_train, y_train=y_train)

In [ ]:
# Set GA parameters
# Define GA parameters
# population: The initial population size (number of individuals)
# ngen: Number of generations to evolve
# cxpb: Crossover probability (probability of mating two individuals)
# mutpb: Mutation probability (probability of mutating an individual)

population = toolbox.population(n=100)  # Increased population size for better diversity
ngen = 50  # Increased number of generations for more thorough exploration
cxpb = 0.8  # Higher crossover probability to encourage exploration
mutpb = 0.1  # Lower mutation probability to reduce randomness

# Run the GA
result_population = algorithms.eaSimple(population, toolbox, cxpb=cxpb, mutpb=mutpb, ngen=ngen, verbose=False)
best_individual = tools.selBest(result_population[0], k=1)[0]
selected_features = [i for i, bit in enumerate(best_individual) if bit == 1]

# Print the chosen features
chosen_features = [feature_names[i] for i in selected_features]
print("Selected Features:")
print(chosen_features)

In [1]:
# Train a model on the selected features
print("Training final model with selected features...")
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]
X_train_selected = pd.DataFrame(X_train_selected, columns=[feature_names[i] for i in selected_features])
X_test_selected = pd.DataFrame(X_test_selected, columns=[feature_names[i] for i in selected_features])

final_model = lgb.LGBMClassifier(
    random_state=42,  # Ensures reproducibility by setting a fixed random seed
    learning_rate=0.10078188554336627,  # Controls the step size during optimization
    num_leaves=62,  # Maximum number of leaves in one tree
    max_depth=12,  # Maximum depth of a tree to prevent overfitting
    n_estimators=289,  # Number of boosting iterations (trees)
    boosting_type='gbdt',  # Gradient Boosting Decision Tree (default boosting type)
    class_weight='balanced',  # Automatically adjusts weights inversely proportional to class frequencies
    verbose_eval=1,  # Controls verbosity of the training process
    min_child_samples=94,  # Minimum number of data points in a leaf node
    subsample=0.7102007989557683,  # Fraction of samples used for training each tree
    colsample_bytree=0.5061939258896568,  # Fraction of features used for training each tree
    reg_alpha=1.9225920516937184e-7,  # L1 regularization term to prevent overfitting
    reg_lambda=4.6875142019482343e-7  # L2 regularization term to prevent overfitting
)
final_model.fit(X_train_selected, y_train)

Training final model with selected features...


NameError: name 'X_train' is not defined

In [ ]:
# Evaluate the model
y_pred = final_model.predict(X_test_selected)


# Get feature importance
feature_importance = final_model.feature_importances_

# Combine feature names and their importance into a DataFrame for better readability
feature_importance_df = pd.DataFrame({
    'Feature': X_train_selected.columns,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

[LightGBM] [Warning] Unknown parameter: verbose_eval


In [ ]:
# Print the feature importance
print("\nFeature Importance:")
print(feature_importance_df)


Feature Importance:
                   Feature  Importance
0         Destination Port        1913
4             Flow IAT Max        1147
5            Fwd IAT Total        1101
1   Bwd Packet Length Mean         840
3             Flow IAT Std         777
9   Packet Length Variance         537
6              Fwd IAT Std         523
8       Packet Length Mean         510
2    Bwd Packet Length Std         505
7              Fwd IAT Max         482
12     Average Packet Size         445
10          PSH Flag Count         430
11          URG Flag Count         246
14                Idle Max         205
15                Idle Min         139
13    Avg Bwd Segment Size           0


In [ ]:
# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
# Print evaluation metrics
print(f"Final Model Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

Final Model Accuracy: 0.9958
Precision: 0.9958
Recall: 0.9958
F1 Score: 0.9958
Confusion Matrix:
[[166484    740]
 [   661 166703]]


In [ ]:
#Print a detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    167224
           1       1.00      1.00      1.00    167364

    accuracy                           1.00    334588
   macro avg       1.00      1.00      1.00    334588
weighted avg       1.00      1.00      1.00    334588

